In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
df1=pd.read_csv("D:/cdrive/desktop/DIVYA/CAREER/guvi/PROJECT/TOURISM/16-03/debug6.csv")

In [5]:
print((df1 == "\\N").sum())  # Count \N occurrences per column
print(df1.isin(["\\N"]).sum().sum())  #


userid                0
visityear             0
visitMonth            0
visitmodeID           0
attractionid          0
rating                0
attraction            0
AttractionAddress     0
AttractionCityId      0
attractiontypeid      0
contenentid           0
regionid              0
countryid             0
cityid                8
attractiontype        0
visitmode             0
UserCity              8
UserCountry           0
UserRegion            0
UserContinent         0
AttractionCityName    0
AttractionCountry     0
AttractionRegion      0
dtype: int64
16


In [7]:
# Replace "\\N" with NaN (modify df1 in place)
df1['cityid'] = df1['cityid'].replace("\\N", np.nan)

# Drop rows with NaN in 'cityid'
df1 = df1.dropna(subset=['cityid'])

# Reset index
df1 = df1.reset_index(drop=True)

# Verify if "\\N" is gone
print((df1 == "\\N").sum())  # Count \N occurrences per column
print(df1.isin(["\\N"]).sum().sum())  # Count total occurrences

userid                0
visityear             0
visitMonth            0
visitmodeID           0
attractionid          0
rating                0
attraction            0
AttractionAddress     0
AttractionCityId      0
attractiontypeid      0
contenentid           0
regionid              0
countryid             0
cityid                0
attractiontype        0
visitmode             0
UserCity              0
UserCountry           0
UserRegion            0
UserContinent         0
AttractionCityName    0
AttractionCountry     0
AttractionRegion      0
dtype: int64
0


In [9]:
df1["cityid"] = pd.to_numeric(df1["cityid"], errors="coerce").fillna(0).astype(int)

In [11]:
##Checking for null values
df1.isnull().sum().any()

False

In [13]:
##Getting an ida about the duplicates
df1.duplicated().sum()
# 0 indicates that there are no duplicates in the data

3722

In [15]:
##dropping the duplicates
df1=df1.drop_duplicates()

In [17]:
##Getting an ida about the duplicates
df1.duplicated().sum()
##0 indicates there are no duplicate values

0

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
X = df1[
    [
        "visityear", "visitMonth", "attractionid",
        "contenentid", "regionid", "countryid", "cityid",
        "AttractionCityId"
    ]
]
y = df1["visitmodeID"]  # Target variable



In [27]:
# Performing train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 5)

In [29]:
# Creating the classifier
clf = RandomForestClassifier(random_state = 5)

# Fitting the data
clf.fit(X_train, y_train)

# Taking predictions from the model
y_pred = clf.predict(X_test)
y_train_pred = clf.predict(X_train)

In [31]:
# Evaluating the model 
# Testing Performance
print('Testing Accuracy :', np.round(metrics.accuracy_score(y_test, y_pred), 2))
print('Testing Precision :', np.round(metrics.precision_score(y_test, y_pred, average = 'weighted'), 2))
print('Testing Recall :', np.round(metrics.recall_score(y_test, y_pred, average = 'weighted'), 2))
print('Testing F1 Score :', np.round(metrics.f1_score(y_test, y_pred, average = 'weighted'), 2))

# Training Performance
print('\nTraining Accuracy :', np.round(metrics.accuracy_score(y_train, y_train_pred), 2))
print('Training Precision :', np.round(metrics.precision_score(y_train, y_train_pred, average = 'weighted'), 2))
print('Training Recall :', np.round(metrics.recall_score(y_train, y_train_pred, average = 'weighted'), 2))
print('Training F1 Score :', np.round(metrics.f1_score(y_train, y_train_pred, average = 'weighted'), 2))

Testing Accuracy : 0.43
Testing Precision : 0.41
Testing Recall : 0.43
Testing F1 Score : 0.42

Training Accuracy : 0.89
Training Precision : 0.89
Training Recall : 0.89
Training F1 Score : 0.89


In [ ]:
##model is overfitting

In [33]:
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

In [35]:
X = df1[
    [
        "visityear", "visitMonth", "attractionid",
        "contenentid", "regionid", "countryid", "cityid",
        "AttractionCityId"
    ]
]
y = df1["visitmodeID"]  # Target variable



In [37]:
# One-Hot Encode Categorical Features
X = pd.get_dummies(X, drop_first=True)  # Avoid multicollinearity

In [39]:
# Encode Target Variable
le = LabelEncoder()
y = le.fit_transform(y)  # Convert target labels into numerical values


In [47]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Get Number of Classes
num_classes = len(np.unique(y))

# Initialize XGBoost Classifier
model = XGBClassifier(
    n_estimators=200,     
    max_depth=6,          
    learning_rate=0.05,   
    reg_lambda=1.0,       
    reg_alpha=0.5,        
    subsample=0.8,        
    colsample_bytree=0.8, 
    random_state=42,
    objective="multi:softmax",  # Use softmax for multi-class classification
    num_class=num_classes      
)

# Set evaluation metric manually
model.set_params(eval_metric="mlogloss")

# Train Model
model.fit(
    X_train, y_train, 
    eval_set=[(X_test, y_test)], 
    verbose=10  
)

# Predict
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

# Evaluate Model
print("Training Accuracy :", accuracy_score(y_train, y_train_pred))
print("Testing Accuracy  :", accuracy_score(y_test, y_pred))
print("Testing Precision :", precision_score(y_test, y_pred, average='weighted', zero_division=1))
print("Testing Recall    :", recall_score(y_test, y_pred, average='weighted', zero_division=1))
print("Testing F1 Score  :", f1_score(y_test, y_pred, average='weighted', zero_division=1))
import joblib
joblib.dump(model, "D:/cdrive/desktop/DIVYA/CAREER/guvi/PROJECT/TOURISM/16-03/visit_mode_model_16.pkl")

[0]	validation_0-mlogloss:1.58699
[10]	validation_0-mlogloss:1.43838
[20]	validation_0-mlogloss:1.35487
[30]	validation_0-mlogloss:1.30606
[40]	validation_0-mlogloss:1.27561
[50]	validation_0-mlogloss:1.25611
[60]	validation_0-mlogloss:1.24308
[70]	validation_0-mlogloss:1.23408
[80]	validation_0-mlogloss:1.22731
[90]	validation_0-mlogloss:1.22252
[100]	validation_0-mlogloss:1.21895
[110]	validation_0-mlogloss:1.21638
[120]	validation_0-mlogloss:1.21455
[130]	validation_0-mlogloss:1.21319
[140]	validation_0-mlogloss:1.21196
[150]	validation_0-mlogloss:1.21098
[160]	validation_0-mlogloss:1.21032
[170]	validation_0-mlogloss:1.20970
[180]	validation_0-mlogloss:1.20922
[190]	validation_0-mlogloss:1.20895
[199]	validation_0-mlogloss:1.20863
Training Accuracy : 0.5178353658536585
Testing Accuracy  : 0.48089430894308943
Testing Precision : 0.46279882000987493
Testing Recall    : 0.48089430894308943
Testing F1 Score  : 0.42356845730879344


['D:/cdrive/desktop/DIVYA/CAREER/guvi/PROJECT/TOURISM/16-03/visit_mode_model_16.pkl']

In [ ]:
'''
The training accuracy (51.9%) and testing accuracy (48.2%) are close to each other, indicating the model is not overfitting to the training data.
This suggests the model is learning patterns that generalize well to new data.
A testing accuracy of 48.2% is reasonable, given the inherent subjectivity in user preferences.
A precision of 45.1% means that when the model predicts a visit mode, it is fairly reliable.
A recall of 48.2% shows the model is capturing nearly half of the actual visit modes, meaning it is retrieving a significant portion of the true cases.
The F1 score (42.4%) balances both, indicating a moderate but stable classification performance.
'''